In [1]:
using RDatasets, MLJ
iris = dataset("datasets", "iris")

first(iris, 3) |> pretty

┌─────────────┬────────────┬─────────────┬────────────┬─────────────────────── ⋯
│ SepalLength │ SepalWidth │ PetalLength │ PetalWidth │ Species                ⋯
│ Float64     │ Float64    │ Float64     │ Float64    │ CategoricalString{UInt ⋯
│ Continuous  │ Continuous │ Continuous  │ Continuous │ Multiclass{3}          ⋯
├─────────────┼────────────┼─────────────┼────────────┼─────────────────────── ⋯
│ 5.1         │ 3.5        │ 1.4         │ 0.2        │ setosa                 ⋯
│ 4.9         │ 3.0        │ 1.4         │ 0.2        │ setosa                 ⋯
│ 4.7         │ 3.2        │ 1.3         │ 0.2        │ setosa                 ⋯
└─────────────┴────────────┴─────────────┴────────────┴─────────────────────── ⋯


In [63]:
@load KNNClassifier

┌ Info: A model type "KNNClassifier" is already loaded. 
│ No new code loaded. 
└ @ MLJModels /Users/david/.julia/packages/MLJModels/ayF3A/src/loading.jl:43


KNNClassifier(K = 5,
              algorithm = :kdtree,
              metric = Distances.Euclidean(0.0),
              leafsize = 10,
              reorder = true,
              weights = :uniform,) @ 1…44

In [2]:
y, X = unpack(iris, ==(:Species), colname -> true)
first(X, 1) |> pretty

┌─────────────┬────────────┬─────────────┬────────────┐
│ SepalLength │ SepalWidth │ PetalLength │ PetalWidth │
│ Float64     │ Float64    │ Float64     │ Float64    │
│ Continuous  │ Continuous │ Continuous  │ Continuous │
├─────────────┼────────────┼─────────────┼────────────┤
│ 5.1         │ 3.5        │ 1.4         │ 0.2        │
└─────────────┴────────────┴─────────────┴────────────┘


In [3]:
X, y = @load_iris;

In [4]:
for m in models(matching(X, y))
    if m.prediction_type == :probabilistic
        println(rpad(m.name, 30), "($(m.package_name))")
    end
end

AdaBoostClassifier            (ScikitLearn)
BaggingClassifier             (ScikitLearn)
BayesianLDA                   (MultivariateStats)
BayesianLDA                   (ScikitLearn)
BayesianQDA                   (ScikitLearn)
BayesianSubspaceLDA           (MultivariateStats)
ConstantClassifier            (MLJModels)
DecisionTreeClassifier        (DecisionTree)
DummyClassifier               (ScikitLearn)
ExtraTreesClassifier          (ScikitLearn)
GaussianNBClassifier          (NaiveBayes)
GaussianNBClassifier          (ScikitLearn)
GaussianProcessClassifier     (ScikitLearn)
GradientBoostingClassifier    (ScikitLearn)
KNNClassifier                 (NearestNeighbors)
KNeighborsClassifier          (ScikitLearn)
LDA                           (MultivariateStats)
LogisticCVClassifier          (ScikitLearn)
LogisticClassifier            (MLJLinearModels)
LogisticClassifier            (ScikitLearn)
MultinomialClassifier         (MLJLinearModels)
ProbabilisticSGDClassifier    (ScikitLearn)
Ran

In [5]:
@load DecisionTreeClassifier
tree_model = DecisionTreeClassifier()

DecisionTreeClassifier(pruning_purity = 1.0,
                       max_depth = -1,
                       min_samples_leaf = 1,
                       min_samples_split = 2,
                       min_purity_increase = 0.0,
                       n_subfeatures = 0,
                       display_depth = 5,
                       post_prune = false,
                       merge_purity_threshold = 0.9,
                       pdf_smoothing = 0.05,) @ 1…60

In [6]:
tree = machine(tree_model, X, y)

Machine{DecisionTreeClassifier} @ 3…49


In [7]:
train, test = partition(eachindex(y), 0.7, shuffle=true)

([82, 39, 62, 18, 79, 87, 144, 100, 85, 106  …  135, 50, 74, 149, 104, 130, 83, 61, 71, 36], [64, 41, 121, 19, 55, 77, 111, 91, 4, 124  …  42, 45, 137, 23, 35, 142, 26, 80, 37, 92])

In [8]:
fit!(tree, rows=train)

┌ Info: Training Machine{DecisionTreeClassifier} @ 3…49.
└ @ MLJ /Users/david/.julia/packages/MLJ/hcR5D/src/machines.jl:179


Machine{DecisionTreeClassifier} @ 3…49


In [9]:
fitted_params(tree) |> print

(tree_or_leaf = Decision Tree
Leaves: 8
Depth:  5, encoding = Dict("virginica" => 0x03,"setosa" => 0x01,"versicolor" => 0x02))

In [10]:
ŷ = predict(tree, rows=test)
@show ŷ[1]

ŷ[1] = UnivariateFinite(setosa=>0.0161, versicolor=>0.968, virginica=>0.0161)


UnivariateFinite(setosa=>0.0161, versicolor=>0.968, virginica=>0.0161)

We can get the probabilities of the classes given the input in a single vector as follows:

In [11]:
ŷ[1].prob_given_class.vals

3-element Array{Float64,1}:
 0.01612903225806452
 0.9677419354838711 
 0.01612903225806452

we can get

In [12]:
ȳ = predict_mode(tree, rows=test)
@show ȳ[1]
@show mode(ŷ[1])

ȳ[1] = "versicolor"
mode(ŷ[1]) = "versicolor"


CategoricalString{UInt8} "versicolor"

In [13]:
mce = mean(cross_entropy(ŷ, y[test]))
round(mce, digits=4)

0.1238

### Ensembling trees

In [14]:
train_ind, test_ind = partition(eachindex(y), 0.7);


In [15]:
tree_model = DecisionTreeClassifier()

DecisionTreeClassifier(pruning_purity = 1.0,
                       max_depth = -1,
                       min_samples_leaf = 1,
                       min_samples_split = 2,
                       min_purity_increase = 0.0,
                       n_subfeatures = 0,
                       display_depth = 5,
                       post_prune = false,
                       merge_purity_threshold = 0.9,
                       pdf_smoothing = 0.05,) @ 8…65

In [70]:
ensemble_model = EnsembleModel(atom=tree_model, n=100);
ensemble_machine = machine(ensemble_model, X, y)
fit!(ensemble_machine, rows=train_ind)

┌ Info: Training Machine{ProbabilisticEnsembleModel{DecisionTreeClassifier}} @ 5…43.
└ @ MLJ /Users/david/.julia/packages/MLJ/hcR5D/src/machines.jl:179


Machine{ProbabilisticEnsembleModel{DecisionTreeClassifier}} @ 5…43


In [71]:
typeof(ensemble_machine)

Machine{MLJ.ProbabilisticEnsembleModel{DecisionTreeClassifier}}

In [69]:
ensemble_machine.model

MLJ.ProbabilisticEnsembleModel(atom = DecisionTreeClassifier(pruning_purity = 1.0,
                                                             max_depth = -1,
                                                             min_samples_leaf = 1,
                                                             min_samples_split = 2,
                                                             min_purity_increase = 0.0,
                                                             n_subfeatures = 0,
                                                             display_depth = 5,
                                                             post_prune = false,
                                                             merge_purity_threshold = 0.9,
                                                             pdf_smoothing = 0.05,),
                               atomic_weights = Float64[],
                               bagging_fraction = 0.8,
                               rng = Random._GLOBAL_RN

In [72]:
y_hat = predict_mode(ensemble_machine,X);
print("accuracy:  $(mean(y_hat .== y))")

accuracy:  0.9333333333333333

We can evaluate the model using crossvalidation as well

In [65]:
estimates = evaluate!(ensemble_machine, resampling=CV());

Evaluating over 6 folds: 100%[=========================] Time: 0:00:00


In [66]:
typeof(estimates)

NamedTuple{(:measure, :measurement, :per_fold, :per_observation),Tuple{Array{MLJBase.CrossEntropy,1},Array{Float64,1},Array{Array{Float64,1},1},Array{Array{Array{Float64,1},1},1}}}

In [73]:
estimates.per_fold

1-element Array{Array{Float64,1},1}:
 [-6.661338147750938e-16, -6.661338147750938e-16, 0.2470819879295552, 0.2064544281290846, Inf, 0.3042676587627781]

## Ensembling other models

We can try the same with a different model 

In [81]:
knn_model        = KNNClassifier()
ensemble_model   = EnsembleModel(atom=knn_model, n=100)
ensemble_machine = machine(ensemble_model, X, y)
fit!(ensemble_machine, rows=train_ind)
y_hat = predict_mode(ensemble_machine,X);
print("accuracy:  $(mean(y_hat .== y))")


accuracy:  0.88

┌ Info: Training Machine{ProbabilisticEnsembleModel{KNNClassifier}} @ 5…85.
└ @ MLJ /Users/david/.julia/packages/MLJ/hcR5D/src/machines.jl:179


In [83]:
target_scitype(knn_model)

AbstractArray{#s62,1} where #s62<:Finite

In [78]:
using MulticlassPerceptron

In [79]:
knn_model        = MulticlassPerceptronClassifier()
ensemble_model   = EnsembleModel(atom=knn_model, n=100)
ensemble_machine = machine(ensemble_model, X, y)
fit!(ensemble_machine, rows=train_ind)
y_hat = predict_mode(ensemble_machine,X);
print("accuracy:  $(mean(y_hat .== y))")

ErrorException: `atom` has unsupported target_scitype `Unknown`. 

In [80]:
target_scitype(knn_model)

Unknown

In [87]:
using MLJBase

In [88]:
metadata_model

metadata_model (generic function with 1 method)